In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
"""This notebook builds the CSV-like Wikimedia Commons page 
which records the contributions to WLE in Spain in YEAR"""

import os, sys, inspect

try :
    import pywikibot as pb
    from pywikibot import pagegenerators, textlib
    import mwparserfromhell as mwh

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[0:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot import pagegenerators, textlib
    import mwparserfromhell as mwh

import pandas as pd
from io import StringIO
import re
from datetime import datetime, timedelta

In [2]:
YEAR              = 2017
TAG               = 'WLE'
TAG_EXT           = 'Wiki Loves Earth'
CODE_TEMPLATE     = 'lic'

BASE_NAME         = "Commons:Wiki Loves in Spain/{1}/{0}".format(YEAR, TAG_EXT)
LOG_PAGE          = BASE_NAME + '/Log'
BASE_SITE_DB_NAME = "Commons:Wiki Loves Earth 2016 in Spain"
SITE_DB_PAGE      = BASE_SITE_DB_NAME + "/SCI DB"

WLE_CAT           = "Images from {1} {0} in Spain".format(YEAR, TAG_EXT)
WLE_FINALIST_CAT  = "Evaluation of images from {1} {0} in Spain - Final".format(YEAR, TAG_EXT)

START_TIME        = datetime(YEAR, 5, 1, 0, 0, 0)
END_TIME          = datetime(YEAR, 6, 1, 0, 0, 0) + timedelta(hours=1)

commons_site = pb.Site("commons", "commons")

### Image selection

In [3]:
cat_wle = pb.Category(commons_site, WLE_CAT)
gen_wle = pagegenerators.CategorizedPageGenerator(cat_wle)

images_wle = [page.title(withNamespace=False) for page in gen_wle if page.is_filepage()]
len(images_wle)

2492

### Retrieval of the sites of community importance

In [ ]:
# retrieval of the WLE SCI (site of community importance) log
pb.output('Retrieving --> WLE SCI list')
site_list_page = pb.Page(commons_site, SITE_DB_PAGE)
site_list_text = StringIO(site_list_page.text[site_list_page.text.find('\n') + 
                                                      1:site_list_page.text.rfind('\n')])
site_df = pd.read_csv(site_list_text, sep=";",
                      index_col=False,
                      names=["name", "code", "magrama_url", "community",
                            "bio_region", "continent", "min_altitude",
                            "max_altitude", "avg_altitude", "longitude",
                            "latitude", "area", "marine_percentage",
                            "marine_area", "image", "commons_cat", "wikidata_id"])

pb.output('Retrieved --> WLE SCI list')

sites_list = site_df['code'].tolist()

Retrieving --> WLE SCI list
Retrieved --> WLE SCI list


### Image review and recording

In [ ]:
images_df = pd.DataFrame(
    columns=['image_title', 'lic_id', 'uploader', 'uploader_registration', 
             'timestamp', 'date', 'size', 'height', 'width', 'qi', 'finalist'])

uploaders = {}

for image_counter, image in enumerate(images_wle):
    if (image_counter != 0) and (image_counter % 50 == 0) :
        pb.output ('Retrieving --> %d image descriptions downloaded' %(image_counter))

    image_row = {'image_title': None, 
                 'lic_id': '', 
                 'uploader': None,
                 'uploader_registration': None,
                 'timestamp': None,
                 'date': None,
                 'size': 0,
                 'height': 0,
                 'width': 0,
                 'qi': None,
                 'finalist': None}

    page = pb.FilePage(commons_site, image)
    text = page.text
    wikicode = mwh.parse(text)

    image_row["image_title"] = page.title(withNamespace=False)
    
    # upload time management
    creation = page.oldest_revision
    creation_time = creation.timestamp + timedelta(hours=2)
    uploader = creation["user"]
    #print (page.oldest_revision.timestamp)
    if ((creation_time > END_TIME) or (creation_time < START_TIME)) and uploader != 'WikiLovesESBot' :
        print ("Image ({}) out of time".format(image_row["image_title"]))
        continue
    image_row["timestamp"] = creation_time.strftime("%Y-%m-%d %H:%M:%S")
    
    fileinfo = page.latest_file_info
    image_row["size"] = fileinfo['size']
    image_row["height"] = fileinfo['height']
    image_row["width"] = fileinfo['width']
    
    # code management
    wle_templates = [template for template in wikicode.filter_templates() 
                    if template.name.lower().strip() == CODE_TEMPLATE]
    if len(wle_templates) > 0 and wle_templates[0].get(1).value.strip() in sites_list:
        image_row["lic_id"] = wle_templates[0].get(1).value.strip()
    else :
        print ("Image with no or wrong code ({0})".format(image_row["image_title"]))
        
    # date management
    info_templates = [template for template in wikicode.filter_templates() 
                      if template.name.lower().strip() == "information"]
    if len(info_templates) != 0 :
        if not (info_templates[0].has("date") or info_templates[0].has("Date")) :
            print ("Image with no date ({0})".format(image_row["image_title"]))
        else :
            if info_templates[0].has("date"):
                date_info = info_templates[0].get("date").value.strip()
            elif info_templates[0].has("Date"):
                date_info = info_templates[0].get("Date").value.strip()
                
            wikidate = mwh.parse(date_info)
            try :
                # we try to determine whether the date is inside a template
                date_info = wikidate.filter_templates()[0].get(1)
            except :
                # no template
                pass
            date_info = date_info.split(' ')[0]

            num_date_tokens = len(date_info.split('-'))
            if num_date_tokens == 2 :
                date_info = '{}-1'.format(date_info)
            elif num_date_tokens == 1 :
                date_info = '{}-1-1'.format(date_info)

            try :
                datetime_object = datetime.strptime(date_info, '%Y-%m-%d')
                image_row["date"] = date_info
            except :
                print ("Image {0} with wrong date ({1})".format(image_row["image_title"], date_info))
    else :
        print ("Image with no info template ({0})".format(image_row["image_title"]))

    # uploader and uploader registration management
    if uploader != 'WikiLovesESBot' :
        image_row["uploader"] = uploader

        if image_row["uploader"] not in uploaders :
            try:
                # Too old users do not have a user registration time
                user_registration = pb.User(commons_site, title=uploader).registration()
                image_row["uploader_registration"] = user_registration.strftime("%Y-%m-%d")
            except :
                image_row["uploader_registration"] = '2006-01-01'
            uploaders[image_row["uploader"]] = image_row["uploader_registration"]
        else :
            image_row["uploader_registration"] = uploaders[image_row["uploader"]]
    else :
        image_row["uploader"] = re.findall(r"\[(.*)\]", info_templates[0].get("Author").value.strip())[0]
        image_row["uploader_registration"] = '2000-01-01'
        #print(image_row["uploader"])

    # quality status management
    qi_templates = [template for template in wikicode.filter_templates() 
                   if template.name.lower().strip() == "qualityimage"]    
    if len(qi_templates) > 0:
        image_row["qi"] = 'qi'
        
    # finalist status management
    cats = [cat.title(withNamespace=False) for cat in textlib.getCategoryLinks(text)]
    if WLE_FINALIST_CAT in cats:
        image_row["finalist"] = 'finalist'

    images_df = images_df.append(image_row, ignore_index=True)

Image with no or wrong code (A pescar!! patiamarilla zambullendose en el Urumea.jpg)
Image with no or wrong code (A rio revuelto.jpg)


Retrieving --> 50 image descriptions downloaded


Image with no or wrong code (Alcarama Ayuntamiento Yanguas.jpg)
Image with no or wrong code (Alcarama Castillo Yanguas.jpg)
Image with no or wrong code (Alcarama Santa Maria Yanguas.jpg)
Image with no or wrong code (Alduides cementerio.jpg)


Retrieving --> 100 image descriptions downloaded


Image with no or wrong code (Ametlla de Segarra, vinyes de Tardor.jpg)


...

Retrieving --> 2300 image descriptions downloaded


Image with no or wrong code (Tenerife neighbor islands.jpg)
Image with no or wrong code (Tenerife ocean.jpg)
Image with no or wrong code (Tenerife peacocks.jpg)
Image with no or wrong code (Tenerife red sunset.jpg)
Image with no or wrong code (Tenerife ship.jpg)
Image with no or wrong code (Tenerife sunrise.jpg)
Image with no or wrong code (Tenerife sunset.jpg)
Image with no or wrong code (Tres colores.jpg)
Image with no or wrong code (Un paraíso prepirenaico.jpg)


Retrieving --> 2350 image descriptions downloaded


Image with no or wrong code (Valdorba Sto cristo Catalain.jpg)


Retrieving --> 2400 image descriptions downloaded


In [ ]:
images_df.describe(include="all")

### Log creation

In [ ]:
buf = StringIO()
images_df.to_csv(buf, index=None, sep=';', encoding='utf-8', header=False)

db_page = pb.Page(commons_site, LOG_PAGE)
db_page.text = u'<pre>\n' + buf.getvalue() + u'</pre>'
db_page.save("{1} {0} in Spain: Contribution log update".format(YEAR, TAG))